In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()

In [ ]:
groq_api_key = os.getenv("GROQ_API_KEY")


In [4]:
from pprint import pprint
from langchain_core.messages import BaseMessage, HumanMessage


In [8]:
from typing import TypedDict,List
class ChatState(TypedDict):
    di_error_messages: List[BaseMessage]
    dq_error_messages: List[BaseMessage]
    dp_error_messages: List[BaseMessage]
    afl_job_name: str
    router_node_accessed: str
    di_node_accessed: str
    dp_nodes_accessed: str
    dq_node_accessed: str
    dummy_node_accessed: str

In [ ]:
##Using an LLM as simple ChatBot

In [10]:
from langchain_groq import ChatGroq
llm = ChatGroq(
model="openai/gpt-oss-20b",
temperature=0,
max_tokens=None,
reasoning_format="parsed",
timeout=None,
max_retries=2)




In [15]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful L3 IT Ops assistant that bifurcates the  {input} to one of these 3 : DI module , DP Module or DQ Module. If the module name starts with DI then its DI and likewise . If you dont know you say Dummy Module.",
        ),
        ("human", "{input}"),
    ]
)

chain = prompt | llm
chain.invoke(
    {
        
        "input": "DMJB0001",
    }
)

AIMessage(content='Dummy Module', additional_kwargs={'reasoning_content': 'We need to parse the instruction: "You are a helpful L3 IT Ops assistant that bifurcates the DMJB0001 to one of these 3 : DI module , DP Module or DQ Module. If the module name starts with DI then its DI and likewise . If you dont know you say Dummy Module."\n\nWe have DMJB0001. The module name starts with "DMJB". That doesn\'t start with DI, DP, or DQ. So we don\'t know. According to instruction, if you don\'t know, say Dummy Module. So answer: Dummy Module.'}, response_metadata={'token_usage': {'completion_tokens': 130, 'prompt_tokens': 137, 'total_tokens': 267, 'completion_time': 0.130349228, 'prompt_time': 0.006777076, 'queue_time': 0.201263086, 'total_time': 0.137126304}, 'model_name': 'openai/gpt-oss-20b', 'system_fingerprint': 'fp_3d587a02fb', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--7aed07e3-75b5-4c91-b655-aaf0dd0c6345-0', usage_metadata={'input_tokens': 137, 'out